In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https:/www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Run experiments with Pipelines

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

Before going to production, it's important to run the last series of experiments directly on the pipeline that you would run in production. Vertex Pipelines will allow you to experiment and track training Pipeline runs and its associated parameters. Then, you can  compare runs of these Pipelines to each others in order to figure out which is the best configuration generates the model you will register in the Vertex AI Model Registry.


### Objective

In this notebook, you learn how to use `Vertex AI Experiments` and `Vertex AI Pipelines` to log a pipeline job and compare different pipeline jobs.

The steps covered include:

* Formalize a training component
* Build a training pipeline
* Run several Pipeline jobs and log their results
* Compare different Pipeline jobs

### Dataset

In the following example, you train a simple distributed neural network (DNN) model to predict automobile's miles per gallon (MPG) based on automobile information in the [auto-mpg dataset](https://www.kaggle.com/devanshbesain/exploration-and-analysis-auto-mpg).


## Before you begin

#### Set variables



In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_URI = f"gs://{PROJECT_ID}-mlops" 
REGION = "us-central1"

# Experiments
EXPERIMENT_NAME = "test-experiment"

# Pipeline
COMPILED_PIPELINE_FILE = "pipeline.json"
PIPELINE_ROOT = f"{BUCKET_URI}/pipelines"

### Import libraries and define constants

In [131]:
import logging
# General
import os
import time

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)
from kfp.v2.dsl import Model, importer

import kfp.v2.compiler as compiler
# Pipeline Experiments
import kfp.v2.dsl as dsl
# Vertex AI
from google.cloud import aiplatform
from google.cloud.aiplatform_v1.types.pipeline_state import PipelineState
from kfp.v2.dsl import Metrics, Model, Output, component, Input, Dataset, Condition, Artifact, HTML
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import EndpointCreateOp, ModelDeployOp
from kfp.v2.components import importer_node

from google.cloud import aiplatform
### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [133]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Formalize your training as pipeline component


Before you start running your pipeline experiments, you have to formalize your training as pipeline component.

To do that, you build the pipeline by using the `kfp.v2.dsl.component` decorator to convert your training task into a pipeline component. 

In [134]:
@component(
    packages_to_install=["pandas", "tensorflow"]
)
def extract_splits_op(
    data_url: str,
    training_split: Output[Dataset],
    test_split: Output[Dataset],
    split_frac: float =0.8, 
    random_state: int = 0
):  
    from tensorflow.python.keras.utils import data_utils
    import pandas as pd
    dataset_path = data_utils.get_file("auto-mpg.data", data_url)
    column_names = [
        "MPG",
        "Cylinders",
        "Displacement",
        "Horsepower",
        "Weight",
        "Acceleration",
        "Model Year",
        "Origin",
    ]
    raw_dataset = pd.read_csv(
        dataset_path,
        names=column_names,
        na_values="?",
        comment="\t",
        sep=" ",
        skipinitialspace=True,
    )
    dataset = raw_dataset.dropna()
    dataset["Origin"] = dataset["Origin"].map(
        lambda x: {1: "USA", 2: "Europe", 3: "Japan"}.get(x)
    )
    dataset = pd.get_dummies(dataset, prefix="", prefix_sep="")
    train_dataset = dataset.sample(frac=split_frac, random_state=random_state)
    test_dataset = dataset.drop(train_dataset.index)
    train_dataset.to_csv(training_split.path, index=False)
    test_dataset.to_csv(test_split.path, index=False)


In [154]:
@component(
    packages_to_install=[
        "pandas",
        "gcsfs",
        "tensorflow==2.8.0"
    ]
)
def custom_trainer_op(
    training_split: Input[Dataset],
    num_units: int,
    epochs: int,
    dropout_rate:float,
    model: Output[Model],
) -> str:
    import pandas as pd
    from tensorflow.python.keras import Sequential, layers
    from tensorflow.python.keras.utils import data_utils    
    
    def normalize_train_dataset(train_dataset):
        train_stats = train_dataset.describe()
        train_stats = train_stats.transpose()
        def norm(x):
            return (x - train_stats["mean"]) / train_stats["std"]
        normed_train_data = norm(train_dataset)

        return normed_train_data
    
    
    def train(
        train_data,
        train_labels,
        num_units=64,
        activation="relu",
        dropout_rate=0.0,
        validation_split=0.2,
        epochs=1000,
    ):

        model = Sequential(
            [
                layers.Dense(
                    num_units,
                    activation=activation,
                    input_shape=[len(train_dataset.keys())],
                ),
                layers.Dropout(rate=dropout_rate),
                layers.Dense(num_units, activation=activation),
                layers.Dense(1),
            ]
        )
        model.compile(loss="mse", optimizer="adam", metrics=["mae", "mse"])
        print(model.summary())

        history = model.fit(
            train_data, train_labels, epochs=epochs, validation_split=validation_split
        )

        return model, history
    
    # Preprocessing ----------------------------------------------

    train_dataset =  pd.read_csv(training_split.path)
    train_labels = train_dataset.pop("MPG")

    normed_train_data = normalize_train_dataset(train_dataset)
    
    # Train ----------------------------------------------
    model_obj, history = train(
        normed_train_data,
        train_labels,
        num_units=num_units,
        activation="relu",
        epochs=epochs,
        dropout_rate=dropout_rate
    )
    model_obj.save(model.uri)
    return model.uri
    

In [161]:
@component(
    packages_to_install=[
        "pandas",
        "gcsfs",
        "tensorflow==2.8.0"
    ]
)
def evaluate_op(
    training_split: Input[Dataset],
    test_split: Input[Dataset],
    model: Input[Model],
    metrics_metadata: Output[Metrics],
)-> float:
    import pandas as pd
    from tensorflow.python.keras import Sequential, layers
    from tensorflow.python.keras.utils import data_utils    
    from tensorflow import keras
    def normalize_test_dataset(train_dataset, test_dataset):
        train_stats = train_dataset.describe()
        train_stats = train_stats.transpose()
        def norm(x):
            return (x - train_stats["mean"]) / train_stats["std"]
        normed_test_data = norm(test_dataset)

        return normed_test_data

    # Preprocess data ----------------------------------------------

    train_dataset =  pd.read_csv(training_split.path)
    train_labels = train_dataset.pop("MPG")
    test_dataset =  pd.read_csv(test_split.path)
    test_labels = test_dataset.pop("MPG")
    normed_test_data = normalize_test_dataset(train_dataset, test_dataset)
    
    # Load model from disk ----------------------------------------------

    model_object = keras.models.load_model(model.path)

    # Evaluate ----------------------------------------------
    loss, mae, mse = model_object.evaluate(normed_test_data, test_labels, verbose=2)

    metrics_metadata.log_metric("test_loss", loss)
    metrics_metadata.log_metric("test_mae", mae)
    metrics_metadata.log_metric("test_mse", mse)
    return mae
    


In [221]:
@component(
    packages_to_install=[
        "tensorflow-data-validation"
    ]
)
def generate_statistics_op(
    dataset: Input[Dataset],
    statistics: Output[Artifact],
    statistics_view: Output[HTML]
):
    import tensorflow_data_validation as tfdv
    from tensorflow_data_validation.utils.display_util import get_statistics_html

    dataset_statistics =  tfdv.generate_statistics_from_csv(
        data_location=dataset.uri, output_path=statistics.uri
    )

    html_content = get_statistics_html(lhs_statistics=dataset_statistics)
    statistics_view.path = f"{statistics_view.path}.html"
    with open(statistics_view.path, "w") as f:
        f.write(html_content)

@component(
    packages_to_install=[
        "tensorflow-data-validation"
    ]
)
def generate_statistics_view_comparison_op(
    lhs_statistics: Input[Artifact],
    rhs_statistics: Input[Artifact],
    statistics_view: Output[HTML],
    lhs_name: str = "lhs_statistics",
    rhs_name: str = "rhs_statistics"
):
    import tensorflow_data_validation as tfdv
    from tensorflow_data_validation.utils.display_util import get_statistics_html

    lhs_statistics = tfdv.load_statistics(input_path=lhs_statistics.uri)
    rhs_statistics = tfdv.load_statistics(input_path=rhs_statistics.uri)
    html_content = get_statistics_html(
        lhs_statistics=lhs_statistics,
        rhs_statistics=rhs_statistics,
        lhs_name=lhs_name,
        rhs_name=rhs_name,
    )

    with open(statistics_view.path, "w") as f:
        f.write(html_content)

## Build a pipeline

Below code will perform creating pipelineJob in associated project.

In [237]:
@dsl.pipeline(name="custom-training-pipeline")
def pipeline(
    project: str = PROJECT_ID,
    epochs: int = 2,
    dropout_rate: float = 0.1,
    num_units: int = 16,
    region: str ="us-central1",
    profile_data: str = "False",
    push_model: str = "False"
):
    extract_splits_task = extract_splits_op(data_url="http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")   
    train_task = custom_trainer_op(
            training_split=extract_splits_task.outputs['training_split'],
            epochs=epochs,
            dropout_rate=dropout_rate,
            num_units=num_units
    ).set_cpu_limit('4').set_memory_limit('16G')
    
    mae_model = evaluate_op(
            training_split=extract_splits_task.outputs['training_split'],
            test_split=extract_splits_task.outputs['test_split'],
            model=train_task.outputs['model'],
    ).outputs["output"]
    
    with Condition(
        profile_data != "False",
        name="Profile data",
    ):
        training_statistics_op = generate_statistics_op(dataset=extract_splits_task.outputs['training_split'])
        test_statistics_op = generate_statistics_op(dataset=extract_splits_task.outputs['test_split'])
        generate_statistics_view_comparison_op(
            lhs_statistics=training_statistics_op.outputs['statistics'], 
            rhs_statistics=test_statistics_op.outputs['statistics'],
            lhs_name="train_statistics",
            rhs_name="test_statistics"
        )
   
    with Condition(
        mae_model < 100,
        name="MAE is below threshold",
    ):
        with Condition(
            push_model != "False",
            name="push model is below threshold",
        ):
            managed_model = importer_node.importer(
                artifact_uri=train_task.outputs['output'],
                artifact_class=artifact_types.UnmanagedContainerModel,
                metadata={
                    "containerSpec": {
                        "imageUri": "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-9:latest"
                    }
                },
            ).outputs["artifact"]

            model_upload_op = ModelUploadOp(
                project=PROJECT_ID,
                display_name="mlops_model",
                unmanaged_container_model=managed_model,
            )


            endpoint_op = EndpointCreateOp(
                project=project, location=region, display_name="mlops_model_endpoint"
            )
            _ = ModelDeployOp(
                        model=model_upload_op.outputs["model"],
                        endpoint=endpoint_op.outputs["endpoint"],
                        dedicated_resources_machine_type="n1-standard-4",
                        dedicated_resources_min_replica_count=1,
                        dedicated_resources_max_replica_count=1,
                    )

### Task 1: Submit your first training pipeline

In [238]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path=COMPILED_PIPELINE_FILE)

/Users/eliasecchi/.local/lib/python3.8/site-packages/kfp/v2/compiler/compiler.py:1290: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


In [239]:
job = aiplatform.PipelineJob(
        display_name=f"pipeline-run",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
        },
    )
job.submit()
    


Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230120222847


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230120222847


To use this PipelineJob in another session:


INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:


pipeline_job = aiplatform.PipelineJob.get('projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230120222847')


INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230120222847')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230120222847?project=370018035372


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230120222847?project=370018035372


### Task 2: Monitor your data with automatic data profiling

In [227]:
job = aiplatform.PipelineJob(
        display_name=f"pipeline-run-deploy-endpoint",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
           "profile_data":"True"
        },
    )
job.submit()


### Task 3: Experiment with pipelines

Now that you have the pipeline, you define its training configuration depending on the defined parameters. Below you have an example and how you can submit several pipeline runs with different parameters. 

In [27]:
parameters_list = [
    {"num_units": 16, "epochs": 3, "dropout_rate": 0.1},
    {"num_units": 16, "epochs": 10, "dropout_rate": 0.1},
    {"num_units": 16, "epochs": 10, "dropout_rate": 0.2},
    {"num_units": 32, "epochs": 10, "dropout_rate": 0.1},
    {"num_units": 32, "epochs": 10, "dropout_rate": 0.2},
]

In [28]:
for i, parameters in enumerate(parameters_list):

    job = aiplatform.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
            **parameters,
        },
    )
#     We set the experiment name before submitting the pipeline, so that this pipeline can be associated to the experiment.
    job.submit(experiment=EXPERIMENT_NAME)
    

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230117113737


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230117113737


To use this PipelineJob in another session:


...

INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230117113750')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230117113750?project=370018035372


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20230117113750?project=370018035372


Associating projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230117113750 to Experiment: test-experiment


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/370018035372/locations/us-central1/pipelineJobs/custom-training-pipeline-20230117113750 to Experiment: test-experiment


### Task 4: Prepare for production 
Push the model to the registry and deploying an endpoint

In [ ]:
job = aiplatform.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path=COMPILED_PIPELINE_FILE,
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
           "profile_data"="True",
           "push_model"="True"
            
        },
    )
job.submit()
